
# MTH3302 : Méthodes probabilistes et statistiques pour l'I.A.

Jonathan Jalbert<br/>
Professeur adjoint au Département de mathématiques et de génie industriel<br/>
Polytechnique Montréal<br/>

Ce TD a été développé avec l'aide précieuse de :
- Sanae Lofti, candidate à la maîtrise,
- Amine Bellahsen, candidat à la maîtrise.<br/>

Tous les deux étaient inscrits au cours à l'automne 2018.

---


# TD 8 : Filtre anti-spam 
___

### Description

Dans ce travail dirigé, vous aurez l'occasion de développer un filtre anti-spam basé sur la classification bayésienne naïve. Que la présence ou l'absence de certains mots sera considérée comme variable explicative.

### Données

Les données exploitées dans ce TD correspondents à des messages électroniques authentiques d'un employé de la compagnie Enron. Vous pouvez télécharger le jeux de données à partir du site web du cours. Le fichier doit être décompressé dans un dossier nommé *data* du répertoire courant de votre calepin Jupyter. 

Notez que les messages électroniques de 158 employés de la compagnie Enron ont été récupérés par la Federal Energy Regulatory Commission pendant la commission d'enquête qui a eu lieu après l'effondrement de la compagnie. Dans ce TD, nous n'utilisons que les messages d'un seul employé. Vous pouvez récupérer le jeu de données entier à l'adresse suivante https://www.cs.cmu.edu/~./enron/enron_mail_20150507.tar.gz.



### Sommaire:
___


[Préliminaires](#unit0)  
[Exercice 1](#unit1)  
[Exercice 2](#unit2)  
[Exercice 3](#unit3)  


In [1]:
using StatsBase, Statistics, DataFrames

# Préliminaires  <a id = "unit0" > </a > 


Les codes de cette section permettent de traiter les fichiers textes correspondant aux messages électroniques avant de pouvoir répondre aux questions.

### Construction des ensembles d'entraînement et de test

Les messages électroniques de l'utilisateur nommé *Enron1* sont répartis dans un dossier *ham* et un dossier *spam*. La cellule de code permet d'extraire aléatoirement des courriels et des pourriels pour l'ensemble d'entraînement. On prend le $2/3$ des messages pour constituer l'échantillon d'entraînement.

Le dictionnaire ``TrainSet`` contient deux champs :
- *:Ham* qui contient la liste des fichiers textes correspondants aux courriels de l'ensemble d'entraînement. 
- *:Spam* qui contient la liste des fichiers textes correspondants aux pourriels de l'ensemble d'entraînement.

Le dictionnaire ``TestSet`` contient les mêmes deux champs mais pour l'ensemble de test.

In [2]:
# Récupération des noms de fichier de tous les hams
filesdir = "../Exercices/Ressources/enron1/ham/"
filename_ham = filesdir.*readdir(filesdir)

# Récupération des noms de fichier de tous les spams
filesdir = "../Exercices/Ressources/enron1/spam/"
filename_spam = filesdir.*readdir(filesdir)

# nombre de fichiers pour contruire l'ensemble d'entraînement des hams
n = Int(round(2/3*length(filename_ham)))

# sélection aléatoire des fichiers de l'ensemble d'entraînement
ham_train = Array{String}(undef,n)
sample!(filename_ham, ham_train, replace=false, ordered=false)
ham_test = setdiff(filename_ham, ham_train)


# nombre de fichiers pour contruire l'ensemble d'entraînement des spams
n = Int(round(2/3*length(filename_spam)))

# sélection aléatoire des fichiers de l'ensemble d'entraînement
spam_train = Array{String}(undef,n)
sample!(filename_spam, spam_train, replace=false, ordered=false)
spam_test = setdiff(filename_spam, spam_train)


# Sauvegarder les ensembles d'entraînement et de test dans des dictionnaires
TrainSet = Dict( :Ham => ham_train, :Spam => spam_train)
TestSet = Dict( :Ham => ham_test, :Spam => spam_test)


Dict{Symbol,Array{String,1}} with 2 entries:
  :Ham  => ["../Exercices/Ressources/enron1/ham/0001.1999-12-10.farmer.ham.txt"…
  :Spam => ["../Exercices/Ressources/enron1/spam/0018.2003-12-18.GP.spam.txt", …

## Dénombrement des mots contenus dans les courriels

Cette cellule permet de dénombrer le nombre de courriels où chaque mot est présent. Le dictionnaire ``ham_wordcounts`` indique le nombre de courriels dans lesquels cahque mot apparaît.

La fonction *wordlisting* prend en entrée le chemin d'accès d'un fichier texte et sort la liste de mots présents du fichier. Si un mot apparaît plus d'une fois dans le fichier, la fonction ne sort que la présence de ce mot, pas la quantité de fois où il apparaît.

La fonction *wordcounting* prend en entrée une matrice de liste de mots. La fonction dénombre le nombre de lignes où le mot apparaît.

In [3]:
"""
wordlisting : Cette fonction transforme un fichier texte en une liste de mots. Le nombre de fois que
              les mots apparaîssent ne sont pas répertorié.
Input: filename::String le chemin du fichier texte 
Output: Array{Sting} Liste des mots contenus dans le fichier texte
"""
function wordlisting(filename::String)
    
    f = read(filename, String)
    text = replace(f, r"[0123456789]" => "")
    words = split(text, r"\W+")
    filter!(x -> length(x) > 1, words)
    wordlist = unique(words)
    
end

"""
wordcounting : À partir d'un Array of Array contenant la liste des mots d'un ensemble de fichiers texte, la fonction 
               retourne le nombre de fois où chaque mot est présent.
Input: Un array correspondant à la liste des mots pour chacun des fichiers texte 
Output: Dictionnaire compilant le nombre de ligne dans lesquelles chacun des mots apparaît.
"""
function wordcounting(A::Array{Array{SubString{String},1},1})
   
    words = vcat(A...)

    wordcounts = Dict{String,Int64}()

    for word in words
        wordcounts[word]=get(wordcounts, word, 0) + 1
    end
    
    return wordcounts
    
end

wordcounting

In [4]:
ham_wordlist = wordlisting.(TrainSet[:Ham])
ham_wordcounts = wordcounting(ham_wordlist)

Dict{String,Int64} with 12895 entries:
  "improvemnts" => 1
  "corbally"    => 1
  "henry"       => 5
  "gerdman"     => 1
  "rises"       => 1
  "hampshire"   => 4
  "tnpc"        => 1
  "progression" => 1
  "rhonda"      => 8
  "msnbc"       => 1
  "gathered"    => 3
  "aldano"      => 1
  "acton"       => 67
  "underground" => 1
  "november"    => 107
  "backup"      => 4
  "caught"      => 1
  "stress"      => 2
  "rectified"   => 1
  "annals"      => 1
  "premature"   => 1
  "fountain"    => 1
  "hicks"       => 1
  "package"     => 22
  "qnec"        => 1
  ⋮             => ⋮

## Dénombrement des mots contenus dans les pourriels

Cette cellule permet de dénombrer le nombre de courriels où chaque mot est présent. Le dictionnaire ``spam_wordcounts`` indique le nombre de courriels dans lesquels cahque mot apparaît.

Les fonctions *wordlisting* et *wordcounting* sont utilisées.

In [5]:
spam_wordlist = wordlisting.(TrainSet[:Spam])
spam_wordcounts = wordcounting(spam_wordlist)

Dict{String,Int64} with 30555 entries:
  "oblique"        => 1
  "rpiebkwgu"      => 1
  "eowrsmi"        => 1
  "vvmiaml"        => 1
  "null"           => 2
  "grp"            => 2
  "staphylococcus" => 1
  "ucewn"          => 1
  "duma"           => 1
  "inattentive"    => 1
  "bumbum"         => 1
  "melons"         => 1
  "henry"          => 2
  "partl"          => 2
  "aslpp"          => 1
  "anaerobic"      => 1
  "corrodible"     => 2
  "hampshire"      => 1
  "youlaagan"      => 1
  "brandt"         => 1
  "maggotadvances" => 1
  "vrmtk"          => 1
  "rhonda"         => 2
  "il"             => 9
  "gathered"       => 16
  ⋮                => ⋮

# Exercice 1 <a id = "unit1" > </a > 

Considérez le mot ***http*** comme variable explicative pour classer les messages électroniques en courriels et pourriels.

### a) Si un message contient le mot ***http***, quelle est la probabilité que ce message soit un pourriel ?

In [6]:
n01 = spam_wordcounts["http"]
n11 = ham_wordcounts["http"]
n₀ = size(TrainSet[:Spam], 1)
n₁ = size(TrainSet[:Ham], 1)
n = n₀ + n₁

3448

In [7]:
q_spam = (n01 + 1)*(n₀ + 1)/((n₀ + 2)*(n +2))
q_ham = (n11 + 1)*(n₁ + 1)/((n₁ + 2)*(n +2))
c = q_spam + q_ham

q_spam/c

0.7691259734542166

### b) Si un message ne contient pas le mot ***http***, quelle est la probabilité que ce message soit un pourriel ?

In [8]:
q_spam = (n₀ - n01 + 1)*(n₀ + 1)/((n₀ + 2)*(n +2))
q_ham = (n₁ - n11 + 1)*(n₁ + 1)/((n₁ + 2)*(n +2))

c = q_spam + q_ham

q_spam/c

0.22453488972525706

### c) Filtrez tous les courriels de l'échantillon de test. Quelle est la proportion de courriels classés comme pourriels?

In [9]:
nb_ham_test = size(TestSet[:Ham],1)
nb_pourriels = 0
for i in 1:nb_ham_test
    X = any(wordlisting(TestSet[:Ham][i]) .== "http")
    if X
        nb_pourriels = nb_pourriels + 1
    end
end

In [10]:
nb_pourriels

40

### d) Filtrez tous les pourriels de l'échantillon de test. Quelle est la proportion de pourriels classés comme pourriels?

In [11]:
nb_spam_test = size(TestSet[:Spam],1)
nb_pourriels = 0
for i in 1:nb_spam_test
    X = count(wordlisting(TestSet[:Spam][i]) .== "http") > 0
    if X
        nb_pourriels = nb_pourriels + 1
    end
end
nb_pourriels/nb_spam_test

0.312

# Exercice 2 <a id = "unit2" > </a > 

Considérez les mots ***http*** et ***enron*** comme variables explicatives pour classer les messages électroniques en courriels et pourriels.

In [30]:
function modelid(M::Array{Any,2},γ)
    
    @assert length(γ)==length(M[1])
    
    ind = 1
    
    while M[ind] != γ
        ind +=1
    end
    
    return M[ind, 2]
        
end

# Énumration de tous les modèles possibles
p = 2
itr = reverse.(Iterators.product(fill(0:1,p)...))[:]

M = BitArray[]
for i in eachindex(itr)
   push!(M, [itr[i]...] .== 1)
end
prop_ham = [M BitArray(undef, 4,1)]

4×2 Array{Any,2}:
 Bool[0, 0]  false
 Bool[0, 1]  false
 Bool[1, 0]  false
 Bool[1, 1]  false

In [33]:
modelid(prop, [1,1])

false

### a) Si un message contient les mots ***http*** et ***enron***, quelle est la probabilité que ce message soit un pourriel ?

In [14]:
n02 = in("enron", keys(spam_wordcounts)) ? spam_wordcounts["enron"] : 0
n12= ham_wordcounts["enron"]

975

In [57]:
q_spam = (n01 + 1)*(n02+1)*(n₀ + 1)/((n₀ + 2)^2*(n+2))
q_ham = (n11 + 1)*(n12+1)*(n₁ + 1)/((n₁ + 2)^2*(n+2))
c = q_spam + q_ham
q_spam/c

0.00827678029918685

### b) Si un message ne contient pas les mots ***http*** et ***enron***, quelle est la probabilité que ce message soit un pourriel ?

In [37]:
q_spam = (n₀ - n01 + 1)*(n₀ - n02+1)*(n₀ + 1)/((n₀ + 2)^2*(n+2))
q_ham = (n₁ - n11 + 1)*(n₁ - n12+1)*(n₁ + 1)/((n₁ + 2)^2*(n+2))
c = q_spam + q_ham
prop[1,2] = q_spam/c < .5

true

### c) Si un message contient le mot ***http*** mais ne contient pas le mot ***enron***, quelle est la probabilité que ce message soit un pourriel ?

In [38]:
q_spam = (n01 + 1)*(n₀ - n02+1)*(n₀ + 1)/((n₀ + 2)^2*(n+2))
q_ham = (n11 + 1)*(n₁ - n12+1)*(n₁ + 1)/((n₁ + 2)^2*(n+2))
c = q_spam + q_ham
prop[3,2] = q_spam/c < .5

false

### d) Si un message ne contient pas le mot ***http*** mais contient le mot ***enron***, quelle est la probabilité que ce message soit un pourriel ?

In [39]:
q_spam = (n₀ - n01 + 1)*(n02+1)*(n₀ + 1)/((n₀ + 2)^2*(n+2))
q_ham = (n₁ - n11 + 1)*(n12+1)*(n₁ + 1)/((n₁ + 2)^2*(n+2))
c = q_spam + q_ham
prop[2,2] = q_spam/c < .5

true

### e) Filtrez tous les courriels de l'échantillon de test. Quelle est la proportion de courriels classés comme pourriels ?

In [42]:
nb_ham_test = size(TestSet[:Ham],1)
nb_pourriels = 0
for i in 1:nb_ham_test
    X = [0,0]
    X[1] = any(wordlisting(TestSet[:Ham][i]) .== "http")
    X[2] = any(wordlisting(TestSet[:Ham][i]) .== "enron")
    if !modelid(prop, X)
        nb_pourriels = nb_pourriels + 1
    end
end
nb_pourriels/nb_ham_test

0.01715686274509804

### f) Filtrez tous les pourriels de l'échantillon de test. Quelle est la proportion de pourriels classés comme pourriels ?

In [48]:
nb_spam_test = size(TestSet[:Spam],1)
nb_pourriels = 0
for i in 1:nb_spam_test
    X = [0,0]
    X[1] = any(wordlisting(TestSet[:Spam][i]) .== "http")
    X[2] = any(wordlisting(TestSet[:Spam][i]) .== "enron")
    if !modelid(prop, X)
        nb_pourriels = nb_pourriels + 1
    end
end
nb_pourriels/nb_spam_test

0.312

# Exercice 3 <a id = "unit3" > </a > 

Pour cet exercice, nous utiliserons les mots les plus discriminant pour filtrer les messages. Pour identifier les mots les plus discriminants, l'information conjointe entre les mots et le classement est utilisée. L'information conjointe est une notion que nous verrons dans le dernier chapitre du cours concernant la théorie de l'information.


L'exécution de la première cellule de code de cette section vous donnera la variable ``discr_words``. Le premier mot de cette liste de mot correspond au mot le plus discriminant pour classer les messages en courriels et pourriels. Le deuxième mot de la liste est le second mot le plus discriminant et ainsi de suite.

In [45]:

struct BagOfWords
    n₀::Int               # Nombre de courriels de l'ensemble d'entraînement
    n₁::Int               # Nombre de pourriels de l'ensemble d'entraînement 
    ham_wordlist::Dict    # Occurrence des mots dans les hams
    spam_wordlist::Dict # Occurrence des mots dans les spams
end

function wordoccurrences(word::T,B::BagOfWords) where T<:AbstractString
    
    ham_wordcounts = B.ham_wordlist
    spam_wordcounts = B.spam_wordlist

    if haskey(ham_wordcounts, word)
        n10 = ham_wordcounts[word]
    else
        n10 = 0
    end

    if haskey(spam_wordcounts, word)
        n11 = spam_wordcounts[word]
    else
        n11 = 0
    end
    
    n = [n10, n11]
    
    return n
    
end

function mutualInformation(word::T,B::BagOfWords) where T<:AbstractString
   
    α = B.n₁ / (B.n₀ + B.n₁)
    
    n = wordoccurrences(word,B)
    
    θ₀₁ = (n[1]+1) / (B.n₀+2)
    θ₁₁ = (n[2]+1) / (B.n₁+2)
    
    θ₁ = (1-α)*θ₀₁ + α*θ₁₁ 
    
    I_mat = [ (1-α)*(1-θ₀₁)*log( (1-θ₀₁)/(1-θ₁) ), (1-α)*θ₀₁*log( θ₀₁/θ₁ ),
        α*(1-θ₁₁)*log( (1-θ₁₁)/(1-θ₁) ), α*θ₁₁*log( θ₁₁/θ₁ )  ]
    
    I = sum(I_mat)
    
    return I
    
end


wordBag = BagOfWords(length(TrainSet[:Ham]),length(TrainSet[:Spam]),ham_wordcounts,spam_wordcounts)

filenames = vcat(TrainSet[:Ham],TrainSet[:Spam])

wordlist = wordlisting.(filenames)

words = unique(vcat(wordlist...))

I = Float64[]

for word in words
   push!(I, mutualInformation(word,wordBag)) 
end

indperm = sortperm(I,rev=true)

discr_words = words[indperm]

37103-element Array{SubString{String},1}:
 "enron"       
 "cc"          
 "hpl"         
 "daren"       
 "gas"         
 "http"        
 "forwarded"   
 "pm"          
 "thanks"      
 "hou"         
 "ect"         
 "subject"     
 "meter"       
 ⋮             
 "environments"
 "fans"        
 "cream"       
 "sailing"     
 "hampshire"   
 "requests"    
 "county"      
 "customer"    
 "system"      
 "young"       
 "previous"    
 "users"       

## a) En prenant les 10 mots les plus discriminant, quelle est la proportion de courriels de l'ensemble de test qui sont classés comme pourriels ?

In [58]:
function factor(word, exists)
    n0i = in(word, keys(spam_wordcounts)) ? spam_wordcounts[word] : 0
    n1i= ham_wordcounts[word]
    
    if exists
        q_spam = (n0i + 1)/(n₀ + 2)
        q_ham = (n1i + 1)/(n₁ + 2)
        return [q_ham, q_spam]
    else
        q_spam = (n₀ - n0i + 1)/(n₀ + 2)
        q_ham = (n₁ - n1i + 1)/(n₁ + 2)
        return [q_ham, q_spam]
    end
end
function prob_spam(M)
    factors = []
    for i in 1:10
        factors[i] = factor(discr_words[i], M[i] == 1)
    end
    
    q_spam = (n₀ + 1)/(n+2)
    for i in 1:10
        q_spam = q_spam * factors[i][2]
    end
    
    q_ham = (n₁ + 1)/(n+2)
    for i in 1:10
        q_ham = q_ham * factors[i][1]
    end
    
    return q_spam/(q_spam+qham)
end
        
prob_spam([1,0,0,0,0,0,0,0,0,0])

BoundsError: BoundsError: attempt to access 0-element Array{Any,1} at index [1]

## b) En prenant les 10 mots les plus discriminant, quelle est la proportion de pourriels de l'ensemble de test qui sont classés comme pourriels ?

## c) En prenant les 100 mots les plus discriminant, quelle est la proportion de courriels de l'ensemble de test qui sont classés comme pourriels ?

## d) En prenant les 100 mots les plus discriminant, quelle est la proportion de pourriels de l'ensemble de test qui sont classés comme pourriels ?

## e) Quel serait le nombre idéal de mots discriminant qu'il faudrait prendre ?